In [12]:
import pandas as pd
import numpy as np
import os
from scipy.cluster import hierarchy as hc
import matplotlib
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats
import math
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
warnings.filterwarnings(action='ignore')

matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

# Load data

In [13]:
dir_root = '../../../../../'
current_dir = os.getcwd()
dir_master = os.path.join(dir_root, '01. Data','00. master_data')
dir_processed = os.path.join(dir_root, '01. Data','02. 재무지표2', '01. preprocessed')
dir_result = os.path.join(dir_root, '04. 상관분석','02. 재무지표2','02. result')

In [14]:
y = '투자비 대비 매출 (Rev/GI)s'

# 탐색기에서 파일명 만들 때, "/"사용 불가하여 처리하는 부분
y_file_name = y.replace("/","")
c1 = 'Accenture'
c2 = 'Infosys'
c3 = 'TCS'
c4 = 'Cognizant'

In [15]:
outperform = [c1, c2, c3]
lowperform = [c4]

In [16]:
target_out = [s+'_show' for s in outperform]
target_low = [s+'_show' for s in lowperform]
target_col = [s+'_show' for s in outperform+lowperform]

In [17]:
# dir_result_all = os.path.join(dir_result, 'All_ITS', y)
dir_result_all = os.path.join(dir_result, 'All_ITS', y_file_name)

In [18]:
df = pd.read_excel(os.path.join(dir_result_all, 'corr_y_x_over7.xlsx'))

In [19]:
df.columns

Index(['variable', 'group', 'importance', 'TCS_filtered', 'Infosys_filtered',
       'Cognizant_filtered', 'Accenture', 'Infosys_show', 'TCS_show',
       'Cognizant_show'],
      dtype='object')

# Make index

In [20]:
df['Outperform_avg'] = df[list(set(target_out)&set(df.columns))].mean(axis=1)
df['Gap(Out~Low)'] = np.abs(df['Outperform_avg']-df[target_low[0]])
df['Std(Out)'] = df[list(set(target_out)&set(df.columns))].std(axis=1)

In [21]:
scaler = MinMaxScaler()
df['Gap normed+1'] = pd.DataFrame(scaler.fit_transform(df[['Gap(Out~Low)']])+1)[0]

In [22]:
scaler = MinMaxScaler()
df['Std normed+1'] = pd.DataFrame(scaler.fit_transform(df[['Std(Out)']])+1)[0]

In [23]:
df['index 1'] = df['Gap normed+1']/df['Std normed+1']

In [24]:
scaler = MinMaxScaler()
df['Std normed original'] = pd.DataFrame(scaler.fit_transform(df[['Std(Out)']]))[0]

In [25]:
df['Std normed 0.5'] = df['Std normed original']*0.5 + 0.5
df['1/Std normed'] = 1/df['Std normed 0.5']

In [26]:
df['Index 2'] = 0.6*df['Gap normed+1']+0.4*df['1/Std normed']

In [27]:
df[[s+'_marked' for s in outperform+lowperform]+[s+'_plot' for s in outperform+lowperform]] = np.nan

In [28]:
writer = pd.ExcelWriter(os.path.join(dir_result_all, 'correlation_index.xlsx'), engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save()